## First create separate images from each wsi image, with annotations in the unstained folder. crop the images (total of 24 annotation, so 24 images per whole slide) and save them first in a different folder in same path. Then pad the images and then save them. Then using those padded images, register them to one another (doesn't matter which way since there are only two images)
## the path is in: \\shelter\Kyu\unstain2stain\biomax_images

In [1]:
import numpy as np
import time
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
import pandas as pd
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
from glob import glob
import xml.etree.ElementTree as ET
import openslide
from skimage.measure import label
from skimage.morphology import remove_small_objects, remove_small_holes, binary_closing, disk

In [2]:
from time import time
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %f" %(f.__name__,end-start))
        return r
    return wrapper

In [3]:
stain_src_path = r'\\shelter\Kyu\unstain2stain\biomax_images\stained'
stain_img_path = glob(os.path.join(stain_src_path,'*.ndpi'))
unstain_src_path = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained'
unstain_img_path = glob(os.path.join(unstain_src_path,'*.ndpi'))
save_mask_img_name = [os.path.basename(_) for _ in unstain_img_path]
save_mask_img_name = [_.replace('.ndpi','') for _ in save_mask_img_name]
pth = r'\\shelter\Kyu\unstain2stain\biomax_images\stained\masks'
pth1 = r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks'
pth2 = r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images'

In [ ]:
# use scikit learn label to find connected objects in stained image:

for i in range(len(stain_img_path)):
    wsi_img = openslide.OpenSlide(stain_img_path[i])
    # target_level = wsi_img.get_best_level_for_downsample(1)
    target_level = 4
    wsi_img = wsi_img.read_region(location=(0,0),level=target_level,size=wsi_img.level_dimensions[target_level])
    wsi_ra = (150 < np.array(wsi_img)[:, :, 0]) & (np.array(wsi_img)[:, :, 1] < 210)
    binary_mask = wsi_ra
    binary_mask = binary_closing(binary_mask,footprint = np.ones((15,15)))
    binary_mask = remove_small_objects(binary_mask,20000)
    dspth = os.path.join(pth1, save_mask_img_name[i] + ".png")
    Image.fromarray(binary_mask).save(dspth)
    labeled_binary_mask = label(binary_mask)
    num_labels = np.max(labeled_binary_mask)
    for idx in range(num_labels+1):
        target_tissue = idx
        target_binary_mask = labeled_binary_mask == target_tissue
        ind = np.argwhere(target_binary_mask)
        margin = 0
        bbox = [np.min(ind[:,0]),np.min(ind[:,1]),np.max(ind[:,0]),np.max(ind[:,1])] #[miny,minx,maxy,maxx]
        bboxm = [bbox[0]-margin,bbox[1]-margin,bbox[2]+margin,bbox[3]+margin]
        bwcropm = target_binary_mask[bboxm[0]:bboxm[2],bboxm[1]:bboxm[3]]
        wsi_img_crop = np.array(wsi_img)[bboxm[0]:bboxm[2],bboxm[1]:bboxm[3]]
        dstpth = os.path.join(pth,save_mask_img_name[i] + "_" + str(idx) + ".png")
        Image.fromarray(bwcropm).save(dstpth)
        dstpth1 = os.path.join(pth2,save_mask_img_name[i] + "_" + str(idx) + ".png")
        Image.fromarray(wsi_img_crop).save(dstpth1)

In [4]:
# use xml to find images:
def xml_to_df(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [10]:
# Then input original image and the coord_df to output the mask:
def create_mask_multi_annot(xml_path, image_path, downsample_factor):
    slide = openslide.open_slide(image_path)
    target_level = slide.get_best_level_for_downsample(downsample_factor)
    target_dim = slide.level_dimensions[target_level]
    rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
    mask = np.zeros((target_dim[1],target_dim[0]), dtype = np.uint8)
    coord_df = xml_to_df(xml_path) #use function above
    for idx, row in coord_df.iterrows():
        xx = row.X.astype(float).astype('int32')
        yy = row.Y.astype(float).astype('int32')
        contours = np.array(list(zip(xx,yy)))
        contours = contours/rsf[0]
        mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=255)
    return mask

# # cv2 method:
# # Input mask and create binary mask and then output label of connected regions:
# def create_binary_mask_label(xml_path, image_path):
#     mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 2)
#     binary_mask = mask > 0
#     _, binary_mask_label = cv2.connectedComponents(binary_mask.astype(np.uint8))
#     return binary_mask_label #returns label of connected regions

In [16]:
pth = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\masks'
pth1 = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\entire_binary_mask'
pth2 = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images'
unstain_xml_path = glob(os.path.join(unstain_src_path,'*.xml'))
save_mask_img_name = [os.path.basename(_) for _ in unstain_xml_path]
save_mask_img_name = [_.replace('.xml','') for _ in save_mask_img_name]

In [17]:
for j in range(len(unstain_xml_path)):
    mask = create_mask_multi_annot(xml_path = unstain_xml_path[j], image_path = unstain_img_path[j], downsample_factor = 2)
    tmp_dstpth = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\entire_binary_mask'
    dstpth = os.path.join(tmp_dstpth,str(save_mask_img_name[j]) + ".png")
    Image.fromarray(mask).save(dstpth)

In [4]:
#multiply entire_binary_mask to each image and then crop based on connected objects:
entire_binary_mask_images = glob(os.path.join(pth1,'*.png'))
save_mask_img_name = [os.path.basename(_) for _ in unstain_img_path]
save_mask_img_name = [_.replace('.ndpi','') for _ in save_mask_img_name]

In [ ]:
# Take entire binary mask and label them and crop to make mask for individual label. Read image with downsample = 2 and coordinate should match to multiply them and get cropped image.
pth = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\masks'
pth1 = r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images'
for i in range(len(entire_binary_mask_images)):
    binary_mask = Image.open(entire_binary_mask_images[i])
    labeled_binary_mask = label(np.array(binary_mask))
    num_labels = np.max(labeled_binary_mask)
    wsi_img = openslide.OpenSlide(unstain_img_path[i])
    target_level = wsi_img.get_best_level_for_downsample(2)
    wsi_img = wsi_img.read_region(location=(0,0),level=target_level,size=wsi_img.level_dimensions[target_level])
    wsi_img = np.array(wsi_img)
    for idx in range(num_labels+1):
        target_tissue = idx
        target_binary_mask = labeled_binary_mask == target_tissue
        ind = np.argwhere(target_binary_mask)
        margin = 0
        bbox = [np.min(ind[:,0]),np.min(ind[:,1]),np.max(ind[:,0]),np.max(ind[:,1])] #[miny,minx,maxy,maxx]
        bboxm = [bbox[0]-margin,bbox[1]-margin,bbox[2]+margin,bbox[3]+margin]
        bwcropm = np.array(target_binary_mask)[bboxm[0]:bboxm[2],bboxm[1]:bboxm[3]]
        dstpth = os.path.join(pth,save_mask_img_name[i] + "_" + str(idx) + ".png")
        Image.fromarray(bwcropm).save(dstpth)
        imcropm = wsi_img[bboxm[0]:bboxm[2],bboxm[1]:bboxm[3]]
        dstpth1 = os.path.join(pth1,save_mask_img_name[i] + "_" + str(idx) + ".png")
        Image.fromarray(imcropm).save(dstpth1)

In [ ]:
# function to pad images to same size:
def pad_images_to_same_size(images):
    """
    :param images: sequence of images
    :return: list of images padded so that all images have same width and height (max width and height are used)
    """
    width_max = 0
    height_max = 0
    for img in images:
        h, w = img.shape[:2]
        width_max = max(width_max, w)
        height_max = max(height_max, h)

    images_padded = []
    for img in images:
        h, w = img.shape[:2]
        diff_vert = height_max - h
        pad_top = diff_vert//2
        pad_bottom = diff_vert - pad_top
        diff_hori = width_max - w
        pad_left = diff_hori//2
        pad_right = diff_hori - pad_left
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(255,255,255))
        assert img_padded.shape[:2] == (height_max, width_max)
        images_padded.append(img_padded)

    return images_padded

In [ ]:
pad_images_to_same_size = _time(pad_images_to_same_size)
optical_flow_tvl1 = _time(optical_flow_tvl1)

In [ ]:
# another side job: register each section of \\shelter\Kyu\unstain2stain\biomax_images

#first create registered image of two adjacent images manually:
img_files_path = [_ for _ in os.listdir(r'\\shelter\Kyu\unstain2stain\biomax_images') if _.endswith(".jpg")]
img_files_path_complete = [os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images', x) for x in img_files_path]
img_files_path_1 = [x.replace('.jpg','') for x in img_files_path]

num = int(len(img_files_path)/2) - 1 #idx = 16, or 17th image
num_plus1 = num + 1 #idx = 17, or 18th image
num_minus1 = num - 1 #idx = 15, or 16th image

start = time()

ref_img_path = img_files_path_complete[num]
mov_img_path = img_files_path_complete[num_plus1]
ref_img = np.array(Image.open(ref_img_path))
mov_img = np.array(Image.open(mov_img_path))

ref_img_g = rgb2gray(ref_img)
mov_img_g = rgb2gray(mov_img)
v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
nr, nc = ref_img_g.shape
row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                     indexing='ij')
end = time()

start = time()

mov_img_warp_ra =[]
for i in range(3):
    mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
    mov_img_warp_ra.append(mov_img_warp)

r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
rgb = np.stack([r,g,b],axis=2)
reg_img = Image.fromarray(rgb)
reg_img.save(r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\CoarseRegIM\run4_16xr_jpg_rszfc1_padsz1000\opticalflow_registered_image\\' + str(img_files_path_1[num_plus1]) + '.jpg')

end = time()
print("time it took to register: "+  str(end-start) + " seconds")